# Обучение NER-модели на трансформерах с HuggingFace

Для начала установим библиотеку HuggingFace Transformers:

In [17]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Далее скачаем датасет [`ner_dataset.csv`](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) с Kaggle (или из материалов курса) и поместим его в текущую директорию. Далее загрузим датасет с помощью Pandas:

In [2]:
import pandas as pd
import tensorflow as tf

df = pd.read_csv('ner_dataset.zip',encoding='unicode-escape',compression='zip')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Посмотрим на список уникальных тегов и создадим словари подстановки, которые мы сможем использовать для преобразования тегов в номера классов:

In [19]:
tags = df.Tag.unique()
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

Теперь преобразуем данные к удобному формату. Каждая запись будет содержать `id` предложения, список токенов и соответствующих им классов в виде словаря:

In [4]:
data = []
s,t,id = [],[],0
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(tag2id[row['Tag']])
    else:
        if len(s)>0:
            data.append({ "id" : id, "ner_tags" : t, "tokens" : s })
        s,t = [row['Word']],[tag2id[row['Tag']]]
        id = int(row["Sentence #"][9:])
data.append({ "id" : id, "ner_tags" : t, "tokens" : s })

In [18]:
data[3]

{'id': 4,
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'tokens': ['Police',
  'put',
  'the',
  'number',
  'of',
  'marchers',
  'at',
  '10,000',
  'while',
  'organizers',
  'claimed',
  'it',
  'was',
  '1,00,000',
  '.']}

Для использования предобученной модели трансформера нам необходимо будет использовать токенизатор от этой модели. Загрузим токенизатор и посмотрим, как он работает на наших данных.

Обратите внимание, что `transformers` позволяют нам автоматически загружать токенизатор нужного типа по имени модели. В качестве модели используем DistilBERT, суффикс **uncased** означает, что он будет приводить все слова к нижнему регистру.

In [7]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Посмотрим, как работает токенизатор на каком-нибудь из наших предложений из датасета:

In [61]:
tokenized_input = tokenizer(data[1]["tokens"], is_split_into_words=True)
tokenized_input

{'input_ids': [101, 2945, 1997, 3548, 2730, 1999, 1996, 4736, 2587, 1996, 13337, 2040, 3344, 23562, 2007, 2107, 14558, 2015, 2004, 1000, 5747, 2193, 2028, 9452, 1000, 1998, 1000, 2644, 1996, 20109, 1012, 1000, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Видим, что на выходе токенизатора получаются поля `input_ids` - идентификаторы слов в словаре, и `attention_mask` - маска, показывающая, какие слова в предложении следует учитывать. Модели семейства BERT чаще всего принимают данные в таком формате.

Токенизатору можно передать либо предложение целиком, либо список, уже разбитый на токены - в этому случае мы добавляем `is_split_into_words=True`.

Чтобы преобразовать предложение обратно в токены-слова, используем следующий код:

In [62]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'families',
 'of',
 'soldiers',
 'killed',
 'in',
 'the',
 'conflict',
 'joined',
 'the',
 'protesters',
 'who',
 'carried',
 'banners',
 'with',
 'such',
 'slogan',
 '##s',
 'as',
 '"',
 'bush',
 'number',
 'one',
 'terrorist',
 '"',
 'and',
 '"',
 'stop',
 'the',
 'bombings',
 '.',
 '"',
 '[SEP]']

Можно заметить, что токенизатор добавляет специальные токены `[CLS]` и `[SEP]` в начало и конец последовательности, поэтому токенизированная последовательность скорее всего будет на 2 символа длинее. Посмотрим, всегда ли это так, или есть какие-то другие случаи:

In [9]:
for i,x in enumerate(data):
  tokenized_input = tokenizer(x["tokens"], is_split_into_words=True)
  tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
  if len(tokens)!=len(x["tokens"])+2:
    print(f"{len(tokens)} - {len(x['tokens'])} <- {i}")
    print(tokens)
    print(x['tokens'])
    break

33 - 30 <- 1
['[CLS]', 'families', 'of', 'soldiers', 'killed', 'in', 'the', 'conflict', 'joined', 'the', 'protesters', 'who', 'carried', 'banners', 'with', 'such', 'slogan', '##s', 'as', '"', 'bush', 'number', 'one', 'terrorist', '"', 'and', '"', 'stop', 'the', 'bombings', '.', '"', '[SEP]']
['Families', 'of', 'soldiers', 'killed', 'in', 'the', 'conflict', 'joined', 'the', 'protesters', 'who', 'carried', 'banners', 'with', 'such', 'slogans', 'as', '"', 'Bush', 'Number', 'One', 'Terrorist', '"', 'and', '"', 'Stop', 'the', 'Bombings', '.', '"']


Видим, что токенизатор заменил слово `slogans` на два токена - `slogan` и `##s`. Токены, начинающиеся с `##` означают продолжение предыдущего слова. Такое разбиение на слова может облегчить обработку различных форм одного и того же слова.

In [64]:
tokenizer.convert_ids_to_tokens(tokenizer('hydroxychloroquine')['input_ids'])

['[CLS]', 'hydro', '##xy', '##ch', '##lor', '##o', '##quin', '##e', '[SEP]']

Нам нужно таким образом синхронизировать последовательность токенов в "новой" трансформерной токенизации и последовательность меток тегов. Для этого опишем функцию `add_labels`, которая принимает на вход словарь из нашего датасета, конвертирует его с помощью токенизатора, и добавляет туда поле `labels` с выровненными метками тегов. Предобученная нейросеть умеет принимать на вход именно такой формат данных.

In [10]:
def add_labels(x):
  tk = tokenizer(x["tokens"], is_split_into_words=True)
  tokens = tokenizer.convert_ids_to_tokens(tk['input_ids'])
  i = 0
  res = []
  for t,z in zip(tokens,tk['input_ids']):
    if i<len(x['tokens']) and x['tokens'][i].lower().startswith(t):
      res.append(x['ner_tags'][i])
      i+=1
    else:
      res.append(-100)
  tk['labels'] = res
  return tk

add_labels(data[1])

{'input_ids': [101, 2945, 1997, 3548, 2730, 1999, 1996, 4736, 2587, 1996, 13337, 2040, 3344, 23562, 2007, 2107, 14558, 2015, 2004, 1000, 5747, 2193, 2028, 9452, 1000, 1998, 1000, 2644, 1996, 20109, 1012, 1000, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]}

Создадим датасет для обучения, конвертировав наш датасет `data` с помощью описанной нами функции, и также преобразовав его к объекту типа `datasets.Dataset`. Это позволит нам в дальнейшем оперировать с этим датасетом средствами `transformers`.

In [11]:
import datasets

dataset = datasets.Dataset.from_list([add_labels(x) for x in data])

Для обучения сети в минибатчах нам потребуется дополнять все последовательности до длины максимальной последовательности в рамках минибатча, т.е. совершать padding. Это можно сделать с помощью специального объекта `DataCollatorForTokenClassification`. Укажем ему, что возвращать необходимо тензоры в формате TensorFlow, передав параметр `return_tensors='tf'` (по умолчанию используется формат PyTorch).

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

Теперь собственно создадим модель, которую мы будем обучать. По аналогии с `AutoTokenizer` используем класс для автоматического создания модели по имени. Поскольку мы будем решать задачу классификации токенов, то используем `AutoModelForTokenClassification`, и приставка `TF` означает, что мы будет работать с моделями в TensorFlow (по умолчанию используется PyTorch).

In [13]:
from transformers import create_optimizer, TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(tags) # id2label=id2label, label2id=label2id
)

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForTokenClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferenc

Мы увидим предупреждающий текст, что модель содержит неинициализированные слои и требует обучения перед использованием - это правильно!

Определим параметры обучения, а также оптимизатор для обучения. Здесь мы используем готовую функцию `create_optimizer`, которая создаст правильный оптимизатор. Обратите внимание, что здесь мы можем указать как число обучающих шагов, так и число шагов *разогрева* (`num_warmup_steps`).

In [ ]:
batch_size = 16
num_train_epochs = 3
num_train_steps = (len(data) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)


Подготовим датасет для обучения. Здесь мы будем разбивать датасет на минибатчи, применять функцию паддинга из описанного нами ранее объекта `data_collator`, а также перемешивать датасет.

In [29]:
tf_train_set = model.prepare_tf_dataset(
    dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:717: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


После всех предварительно выполненных нами шагов, сам процесс обучения выгдядит очень просто. Мы компилируем модель с полученным ранее оптимизатором, и запускаем обучение функцией `fit`:

In [30]:
model.compile(optimizer=optimizer)
model.fit(tf_train_set, epochs=3)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Epoch 1/3


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:717: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


2997/2997 [==============================] - 350s 110ms/step - loss: 0.1589
Epoch 2/3
2997/2997 [==============================] - 337s 112ms/step - loss: 0.0946
Epoch 3/3
2997/2997 [==============================] - 342s 114ms/step - loss: 0.0781


Для проверки того, как работает наша модель, подадим ей на вход какое-нибудь предложение. Важно не забывать, что, поскольку наша модель реализована на TensorFlow, то при вызове токенизатора надо передать параметр `return_tensors='tf'`.

Далее применяем модель и получаем двумерный тензор логитов, в котором для каждого токена содержится распределение ненормированных вероятностей его соответствия тому или иному тегу.

In [38]:
text = "I would like to go to Paris with John Doe tomorrow evening. Please book a flight!"
tk = tokenizer(text,return_tensors='tf')
res = model(tk).logits
res.shape

TensorShape([1, 20, 17])

Для получения номеров тегов нам необходимо применить в этому функцию `argmax`:

In [51]:
lbls = tf.argmax(res[0],axis=1).numpy()
lbls

array([ 0,  0,  0,  0,  0,  0,  0,  1,  0,  3, 10,  7, 12,  0,  0,  0,  0,
        0,  0,  0])

Теперь конвертируем последовательность `input_ids` в слова, и напечатаем для каждого слова соответствующий ему тег:

In [60]:
words = tokenizer.convert_ids_to_tokens(tk['input_ids'][0])
for w, lbl in zip(words,lbls):
  print(f"{w} -> {id2tag[lbl]}")

[CLS] -> O
i -> O
would -> O
like -> O
to -> O
go -> O
to -> O
paris -> B-geo
with -> O
john -> B-per
doe -> I-per
tomorrow -> B-tim
evening -> I-tim
. -> O
please -> O
book -> O
a -> O
flight -> O
! -> O
[SEP] -> O


Это успех! Модель успешно распознаёт сущности типа `GEO`, `PER` и `TIM`!